In [ ]:
class AlignCombine(CreatePlots):
    align_button = param.Action(lambda x: x.param.trigger('align_button'), label='Align images')

    def __init__(self, **params):
        super().__init__(**params)
        self.files_to_align = []
        self.wrong_filter = False

    def gather_files(self):
        if len(self.selected_table_widget.selection) < 2:
            return None
        else:
            filters = ['R', 'V', 'B']
            self.files_to_align = []
            for i in self.selected_table_widget.selected_dataframe.index:
                home_dir = '../Data/Raw/'
                file_name = self.data().loc[i, 'filename']
                file_path = home_dir + self.give_file_name(file_name)
                fits_file = fits.open(file_path)
                file_filter = fits_file[0].header['FILTER']
                fits_file.close()
                if file_filter in filters:
                    self.files_to_align.append(file_path)
                else:
                    self.wrong_filter = True
                    return None
            if len(self.files_to_align) < 2:
                return None
            return self.files_to_align

    @param.depends('align_button', watch=True)
    def align_images(self):
        file_list = self.gather_files()
        target = file_list[0]
        # for i in range(1, len(file_list)):
        #

    @param.depends('align_button')
    def align_selection_error(self):
        if len(self.selected_table_widget.selection) < 2 or len(self.files_to_align) < 2:
            message = "You have to select at least two files with R, V or B filters if you want to align images."
            pn.state.notifications.error(message, duration=0)

    @param.depends('align_button')
    def filter_selection_error(self):
        if self.wrong_filter:
            self.wrong_filter = False
            message = "You have to select files with R, V or B filters."
            pn.state.notifications.error(message, duration=0)